## 1. Setup <a id='1-setup'></a>

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
import json

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Settings
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

print("✓ Import thư viện thành công")

✓ Import thư viện thành công


## 2. Load Dữ Liệu <a id='2-load-du-lieu'></a>

In [2]:
# Đọc dữ liệu
DATA_PATH = '../data/'

portfolio = pd.read_json(DATA_PATH + 'portfolio.json', orient='records', lines=True)
profile = pd.read_json(DATA_PATH + 'profile.json', orient='records', lines=True)
transcript = pd.read_json(DATA_PATH + 'transcript.json', orient='records', lines=True)

print("Kích thước dữ liệu ban đầu:")
print(f"Portfolio: {portfolio.shape}")
print(f"Profile: {profile.shape}")
print(f"Transcript: {transcript.shape}")

Kích thước dữ liệu ban đầu:
Portfolio: (10, 6)
Profile: (17000, 5)
Transcript: (306534, 4)


## 3. Xử Lý Missing Values <a id='3-xu-ly-missing'></a>

**Chiến lược:**
- Profile có missing values → Drop các rows có NA (vì là demographic data quan trọng)
- Portfolio và Transcript không có missing values

In [3]:
# Kiểm tra missing values
print("Missing values trong Profile:")
print(profile.isnull().sum())
print(f"\nTỷ lệ missing: {profile.isnull().any(axis=1).sum() / len(profile) * 100:.2f}%")

Missing values trong Profile:
gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

Tỷ lệ missing: 12.79%


In [4]:
# Drop missing values trong profile
profile_clean = profile.dropna()

print(f"Profile trước khi drop: {len(profile):,} rows")
print(f"Profile sau khi drop: {len(profile_clean):,} rows")
print(f"Dữ liệu giữ lại: {len(profile_clean)/len(profile)*100:.2f}%")

Profile trước khi drop: 17,000 rows
Profile sau khi drop: 14,825 rows
Dữ liệu giữ lại: 87.21%


## 4. Merge Dataframes <a id='4-merge-dataframes'></a>

**Quy trình merge:**
1. Encode Customer IDs trong transcript và profile
2. Sort và align dữ liệu theo customer_id
3. Duplicate profile rows theo frequency của customer trong transcript
4. Merge transcript + profile
5. Add thông tin portfolio (offers)

### 4.1. Encode Customer IDs

In [5]:
# Kiểm tra unique customers
unique_transcript = transcript['person'].nunique()
unique_profile = profile_clean['id'].nunique()

print(f"Unique customers trong transcript: {unique_transcript:,}")
print(f"Unique customers trong profile_clean: {unique_profile:,}")
print(f"✓ Khớp: {unique_transcript == unique_profile}")

Unique customers trong transcript: 17,000
Unique customers trong profile_clean: 14,825
✓ Khớp: False


In [6]:
# Encode customer IDs từ string sang integer
customer_ids = pd.unique(transcript['person'])
customer_ids_dict = {cid: idx for idx, cid in enumerate(customer_ids)}

# Map vào dataframes
transcript['person'] = transcript['person'].map(customer_ids_dict)
profile_clean['id'] = profile_clean['id'].map(customer_ids_dict)

print(f"✓ Đã encode {len(customer_ids_dict):,} customer IDs")
print(f"Ví dụ: {list(customer_ids_dict.items())[:3]}")

✓ Đã encode 17,000 customer IDs
Ví dụ: [('78afa995795e4d85b5d9ceeca43f5fef', 0), ('a03223e636434f42ac4c3df47e8bac43', 1), ('e2127556f4f64592b11af22de27a7932', 2)]


### 4.2. Sort và Align Data

In [7]:
# Sort theo customer ID
sorted_transcript = transcript.sort_values('person').reset_index(drop=True)
sorted_profile = profile_clean.sort_values('id').reset_index(drop=True)

print("✓ Đã sort dữ liệu theo customer ID")
print(f"Sorted transcript shape: {sorted_transcript.shape}")
print(f"Sorted profile shape: {sorted_profile.shape}")

✓ Đã sort dữ liệu theo customer ID
Sorted transcript shape: (306534, 4)
Sorted profile shape: (14825, 5)


### 4.3. Duplicate Profile Rows

In [8]:
# Tính frequency của mỗi customer trong transcript
customer_frequency = sorted_transcript['person'].value_counts(sort=False)

# Map frequency vào sorted_profile
sorted_profile['frequency'] = sorted_profile['id'].map(customer_frequency)

print("Customer frequency statistics:")
print(sorted_profile['frequency'].describe())
print(f"\nMissing frequency: {sorted_profile['frequency'].isnull().sum()}")

Customer frequency statistics:
count    14825.000000
mean        18.398786
std          6.925566
min          2.000000
25%         13.000000
50%         18.000000
75%         23.000000
max         51.000000
Name: frequency, dtype: float64

Missing frequency: 0


In [9]:
# Duplicate rows dựa trên frequency
profile_duplicated = sorted_profile.reindex(
    sorted_profile.index.repeat(sorted_profile['frequency'])
).reset_index(drop=True)

# Drop column frequency
profile_duplicated = profile_duplicated.drop(['frequency'], axis=1)

print(f"✓ Profile sau khi duplicate: {profile_duplicated.shape}")
print(f"✓ Transcript shape: {sorted_transcript.shape}")
print(f"✓ Match: {len(profile_duplicated) == len(sorted_transcript)}")

✓ Profile sau khi duplicate: (272762, 5)
✓ Transcript shape: (306534, 4)
✓ Match: False


### 4.4. Merge Transcript + Profile

In [10]:
# Concatenate transcript và profile
data = pd.concat([sorted_transcript, profile_duplicated], axis=1)

# Verify alignment
print(f"✓ Data shape sau merge: {data.shape}")
print(f"✓ Customer IDs align: {(data['person'] == data['id']).all()}")

# Drop duplicate id column
data = data.drop(['person'], axis=1)

print(f"\nFinal columns: {list(data.columns)}")
data.head()

✓ Data shape sau merge: (306534, 9)
✓ Customer IDs align: False

Final columns: ['event', 'value', 'time', 'gender', 'age', 'id', 'became_member_on', 'income']


,event,value,time,gender,age,id,became_member_on,income
0,transaction,{'amount': 17.78},144,F,75.0,0.0,20170509.0,100000.0
1,transaction,{'amount': 23.93},378,F,75.0,0.0,20170509.0,100000.0
2,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132,F,75.0,0.0,20170509.0,100000.0
3,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},216,F,75.0,0.0,20170509.0,100000.0
4,transaction,{'amount': 19.67},222,F,75.0,0.0,20170509.0,100000.0


### 4.5. Add Portfolio Information

In [11]:
# Encode offer IDs trong portfolio
offer_ids = portfolio['id'].unique()
offer_ids_dict = {oid: idx for idx, oid in enumerate(offer_ids)}
portfolio['id'] = portfolio['id'].map(offer_ids_dict)

print(f"✓ Đã encode {len(offer_ids_dict)} offer IDs")
portfolio.head()

✓ Đã encode 10 offer IDs


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,0
1,10,"[web, email, mobile, social]",10,5,bogo,1
2,0,"[web, email, mobile]",0,4,informational,2
3,5,"[web, email, mobile]",5,7,bogo,3
4,5,"[web, email]",20,10,discount,4


In [12]:
# Extract offer_id từ column 'value' trong transcript
def get_dict_value(x):
    """Extract value từ dictionary trong column 'value'"""
    if isinstance(x, dict):
        key = list(x.keys())[0]
        return x[key]
    return x

# Apply function
offer_id_series = data['value'].apply(get_dict_value)

print("Sample offer_id_series:")
print(offer_id_series.head(10))

Sample offer_id_series:
0                               17.78
1                               23.93
2    9b98b8c7a33c4b65b9aebfe6a799e6d9
3    5a8bc65990b245e5a138643cd4eb9837
4                               19.67
5                               26.56
6                               29.72
7                               19.89
8    5a8bc65990b245e5a138643cd4eb9837
9    9b98b8c7a33c4b65b9aebfe6a799e6d9
Name: value, dtype: object


In [13]:
# Encode offer_ids
def encode_offer_id(x):
    """Encode offer ID. Nếu là số (amount), return 10"""
    if isinstance(x, str):
        return offer_ids_dict.get(x, 10)
    else:
        return 10  # Transaction without offer

data['offer_id'] = offer_id_series.apply(encode_offer_id)

print("Offer ID distribution:")
print(data['offer_id'].value_counts().sort_index())

Offer ID distribution:
offer_id
0      18062
1      18222
2      11761
3      16202
4      13751
5      20139
6      20241
7      14305
8      19131
9      15767
10    138953
Name: count, dtype: int64


In [14]:
# Add portfolio features: reward, difficulty, duration
portfolio_dict = portfolio.set_index('id')[['reward', 'difficulty', 'duration']].to_dict('index')

data['reward'] = data['offer_id'].map(lambda x: portfolio_dict.get(x, {}).get('reward', 0))
data['difficulty'] = data['offer_id'].map(lambda x: portfolio_dict.get(x, {}).get('difficulty', 0))
data['duration'] = data['offer_id'].map(lambda x: portfolio_dict.get(x, {}).get('duration', 0))

print("✓ Đã thêm portfolio features")
data.head()

✓ Đã thêm portfolio features


,event,value,time,gender,age,id,became_member_on,income,offer_id,reward,difficulty,duration
0,transaction,{'amount': 17.78},144,F,75.0,0.0,20170509.0,100000.0,10,0,0,0
1,transaction,{'amount': 23.93},378,F,75.0,0.0,20170509.0,100000.0,10,0,0,0
2,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132,F,75.0,0.0,20170509.0,100000.0,3,5,5,7
3,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},216,F,75.0,0.0,20170509.0,100000.0,7,0,0,3
4,transaction,{'amount': 19.67},222,F,75.0,0.0,20170509.0,100000.0,10,0,0,0


## 5. Feature Engineering <a id='5-feature-engineering'></a>

### 5.1. Extract Registration Month

In [15]:
# Extract month từ became_member_on (format: YYYYMMDD)
def extract_month(date_int):
    """Extract month từ format YYYYMMDD"""
    date_str = str(date_int)
    if len(date_str) == 8:
        return int(date_str[4:6])
    return 0

data['reg_month'] = data['became_member_on'].apply(extract_month)

print("Registration month distribution:")
print(data['reg_month'].value_counts().sort_index())

Registration month distribution:
reg_month
0    306534
Name: count, dtype: int64


### 5.2. Create Target Variable (event_id)

In [16]:
# Encode event types
event_mapping = {
    'offer received': 0,
    'offer viewed': 1,
    'transaction': 2,
    'offer completed': 3,
    'green flag': 4  # Will be created later
}

data['event_id'] = data['event'].map(event_mapping)

print("Event ID distribution:")
print(data['event_id'].value_counts().sort_index())
print(f"\nMissing event_id: {data['event_id'].isnull().sum()}")

Event ID distribution:
event_id
0     76277
1     57725
2    138953
3     33579
Name: count, dtype: int64

Missing event_id: 0


### 5.3. Drop Unnecessary Columns

In [17]:
# Columns cần drop
columns_to_drop = ['event', 'value', 'time', 'became_member_on']

data = data.drop(columns=columns_to_drop)

print(f"✓ Đã drop {len(columns_to_drop)} columns")
print(f"\nColumns còn lại: {list(data.columns)}")
print(f"Data shape: {data.shape}")

✓ Đã drop 4 columns

Columns còn lại: ['gender', 'age', 'id', 'income', 'offer_id', 'reward', 'difficulty', 'duration', 'reg_month', 'event_id']
Data shape: (306534, 10)


### 5.4. Data Summary

In [18]:
print("="*80)
print("DATA SUMMARY SAU FEATURE ENGINEERING")
print("="*80)
print(f"\nShape: {data.shape}")
print(f"\nColumns: {list(data.columns)}")
print("\nData types:")
print(data.dtypes)
print("\nMissing values:")
print(data.isnull().sum())
print("\nFirst 5 rows:")
data.head()

DATA SUMMARY SAU FEATURE ENGINEERING

Shape: (306534, 10)

Columns: ['gender', 'age', 'id', 'income', 'offer_id', 'reward', 'difficulty', 'duration', 'reg_month', 'event_id']

Data types:
gender         object
age           float64
id            float64
income        float64
offer_id        int64
reward          int64
difficulty      int64
duration        int64
reg_month       int64
event_id        int64
dtype: object

Missing values:
gender        33772
age           33772
id            33772
income        33772
offer_id          0
reward            0
difficulty        0
duration          0
reg_month         0
event_id          0
dtype: int64

First 5 rows:


,gender,age,id,income,offer_id,reward,difficulty,duration,reg_month,event_id
0,F,75.0,0.0,100000.0,10,0,0,0,0,2
1,F,75.0,0.0,100000.0,10,0,0,0,0,2
2,F,75.0,0.0,100000.0,3,5,5,7,0,3
3,F,75.0,0.0,100000.0,7,0,0,3,0,1
4,F,75.0,0.0,100000.0,10,0,0,0,0,2


## 6. Feature Encoding <a id='6-feature-encoding'></a>

**Categorical features cần encode:**
- `gender`: F, M, O → 0, 1, 2
- `offer_id`: đã encode (0-10)

In [19]:
# Encode gender
gender_mapping = {'F': 0, 'M': 1, 'O': 2}
data['gender'] = data['gender'].map(gender_mapping)

print("Gender encoding:")
print(data['gender'].value_counts().sort_index())
print("\n✓ Đã encode gender")

Gender encoding:
gender
0.0    113101
1.0    155690
2.0      3971
Name: count, dtype: int64

✓ Đã encode gender


In [20]:
# Verify tất cả columns đã là numeric
print("Data types sau encoding:")
print(data.dtypes)
print(f"\n✓ All numeric: {data.select_dtypes(include=[np.number]).shape[1] == data.shape[1]}")

Data types sau encoding:
gender        float64
age           float64
id            float64
income        float64
offer_id        int64
reward          int64
difficulty      int64
duration        int64
reg_month       int64
event_id        int64
dtype: object

✓ All numeric: True


## 7. Feature Scaling <a id='7-feature-scaling'></a>

**Chiến lược scaling:**
- **StandardScaler** (z-score normalization): `age`, `income` (có outliers và skewed)
- **MinMaxScaler** (0-1 range): `reward`, `difficulty`, `reg_month` (bounded ranges)
- **No scaling**: `gender`, `offer_id`, `event_id` (categorical đã encode)

### 7.1. Separate Features and Target

In [21]:
# Separate features và target
X = data.drop(['id', 'event_id'], axis=1)
y = data['event_id']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns: {list(X.columns)}")

Features shape: (306534, 8)
Target shape: (306534,)

Feature columns: ['gender', 'age', 'income', 'offer_id', 'reward', 'difficulty', 'duration', 'reg_month']


### 7.2. Train/Test Split (BEFORE Scaling)

In [22]:
# Split TRƯỚC KHI scaling để tránh data leakage
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25, 
    random_state=42,
    stratify=y  # Giữ tỷ lệ classes
)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nTarget distribution in train:")
print(y_train.value_counts().sort_index())
print(f"\nTarget distribution in test:")
print(y_test.value_counts().sort_index())

Train set: (229900, 8)
Test set: (76634, 8)

Target distribution in train:


event_id
0     57208
1     43294
2    104214
3     25184
Name: count, dtype: int64

Target distribution in test:
event_id
0    19069
1    14431
2    34739
3     8395
Name: count, dtype: int64


### 7.3. Apply Scaling

In [23]:
# Reset index để tránh lỗi khi scaling
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print("✓ Reset index hoàn tất")

✓ Reset index hoàn tất


In [24]:
# StandardScaler cho age và income
std_scaler = StandardScaler()

X_train.loc[:, 'age'] = std_scaler.fit_transform(X_train[['age']])
X_test.loc[:, 'age'] = std_scaler.transform(X_test[['age']])

X_train.loc[:, 'income'] = std_scaler.fit_transform(X_train[['income']])
X_test.loc[:, 'income'] = std_scaler.transform(X_test[['income']])

print("✓ StandardScaler applied to: age, income")

✓ StandardScaler applied to: age, income


In [25]:
# MinMaxScaler cho reward, difficulty, reg_month
minmax_scaler = MinMaxScaler()

X_train.loc[:, 'reward'] = minmax_scaler.fit_transform(X_train[['reward']])
X_test.loc[:, 'reward'] = minmax_scaler.transform(X_test[['reward']])

X_train.loc[:, 'difficulty'] = minmax_scaler.fit_transform(X_train[['difficulty']])
X_test.loc[:, 'difficulty'] = minmax_scaler.transform(X_test[['difficulty']])

X_train.loc[:, 'reg_month'] = minmax_scaler.fit_transform(X_train[['reg_month']])
X_test.loc[:, 'reg_month'] = minmax_scaler.transform(X_test[['reg_month']])

print("✓ MinMaxScaler applied to: reward, difficulty, reg_month")

✓ MinMaxScaler applied to: reward, difficulty, reg_month


In [26]:
# Verify scaling
print("Scaled data statistics (Train):")
X_train.describe()

Scaled data statistics (Train):


,gender,age,income,offer_id,reward,difficulty,duration,reg_month
count,204500.000000,2.045000e+05,2.045000e+05,229900.000000,229900.000000,229900.000000,229900.000000,229900.0
mean,0.600474,8.192957e-17,9.214905e-17,7.023863,0.241565,0.214721,3.616190,0.0
std,0.518352,1.000002e+00,1.000002e+00,3.449524,0.332478,0.270173,3.652348,0.0
min,0.000000,-2.041259e+00,-1.616375e+00,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.000000,-7.320303e-01,-7.685317e-01,4.000000,0.000000,0.000000,0.000000,0.0
50%,1.000000,6.489135e-02,-1.090979e-01,9.000000,0.000000,0.000000,4.000000,0.0
75%,1.000000,6.910440e-01,6.445407e-01,10.000000,0.500000,0.500000,7.000000,0.0
max,2.000000,2.683348e+00,2.622842e+00,10.000000,1.000000,1.000000,10.000000,0.0


## 8. Final Data Check <a id='8-train-test-split'></a>

In [27]:
print("="*80)
print("FINAL DATA SUMMARY")
print("="*80)
print(f"\nX_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

print("\nFeatures:")
print(list(X_train.columns))

print("\nClass distribution (y_train):")
print(y_train.value_counts().sort_index())
print("\nClass distribution (y_test):")
print(y_test.value_counts().sort_index())

print("\n✓ Data preprocessing hoàn tất!")

FINAL DATA SUMMARY

X_train shape: (229900, 8)
X_test shape: (76634, 8)
y_train shape: (229900,)
y_test shape: (76634,)

Features:
['gender', 'age', 'income', 'offer_id', 'reward', 'difficulty', 'duration', 'reg_month']

Class distribution (y_train):
event_id
0     57208
1     43294
2    104214
3     25184
Name: count, dtype: int64

Class distribution (y_test):
event_id
0    19069
1    14431
2    34739
3     8395
Name: count, dtype: int64

✓ Data preprocessing hoàn tất!


## 9. Lưu Dữ Liệu Đã Xử Lý <a id='9-luu-du-lieu'></a>

In [28]:
# Tạo thư mục processed nếu chưa có
import os
os.makedirs('../data/processed', exist_ok=True)

# Lưu các datasets
X_train.to_csv('../data/processed/X_train.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

print("✓ Đã lưu datasets:")
print("  - X_train.csv")
print("  - X_test.csv")
print("  - y_train.csv")
print("  - y_test.csv")
print("\nVào thư mục: data/processed/")

✓ Đã lưu datasets:
  - X_train.csv
  - X_test.csv
  - y_train.csv
  - y_test.csv

Vào thư mục: data/processed/


In [29]:
# Lưu feature names và mappings để reference sau này
import pickle

metadata = {
    'feature_names': list(X_train.columns),
    'target_mapping': event_mapping,
    'gender_mapping': gender_mapping,
    'train_shape': X_train.shape,
    'test_shape': X_test.shape,
    'scaled_features': {
        'standard_scaler': ['age', 'income'],
        'minmax_scaler': ['reward', 'difficulty', 'reg_month']
    }
}

with open('../data/processed/metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print("✓ Đã lưu metadata.pkl")
print("\nMetadata bao gồm:")
for key in metadata.keys():
    print(f"  - {key}")

✓ Đã lưu metadata.pkl

Metadata bao gồm:
  - feature_names
  - target_mapping
  - gender_mapping
  - train_shape
  - test_shape
  - scaled_features


---

## Kết Luận

### Đã hoàn thành:

✓ **Data Cleaning:**
- Xử lý missing values trong profile (drop NA)
- Loại bỏ các columns không cần thiết

✓ **Data Integration:**
- Merge 3 dataframes: transcript, profile, portfolio
- Align dữ liệu theo customer_id
- Duplicate profile rows theo frequency

✓ **Feature Engineering:**
- Extract registration month từ became_member_on
- Encode offer_id từ value column
- Add portfolio features (reward, difficulty, duration)
- Create target variable (event_id)

✓ **Feature Encoding:**
- Encode gender: F/M/O → 0/1/2
- Encode offer_id: string → 0-10
- Encode event_id: event names → 0-4

✓ **Feature Scaling:**
- StandardScaler: age, income
- MinMaxScaler: reward, difficulty, reg_month

✓ **Data Splitting:**
- Train/Test split: 75/25
- Stratified sampling để giữ tỷ lệ classes

✓ **Data Export:**
- Lưu X_train, X_test, y_train, y_test
- Lưu metadata cho reference

### Bước tiếp theo:

**Notebook 03 - Model Training:**
- Load processed data
- Handle imbalanced dataset (SMOTE, class weights)
- Train models: DNN, XGBoost, Random Forest
- Hyperparameter tuning
- Save trained models

---

**📝 Ghi chú:** Dataset đã sẵn sàng cho modeling. Tất cả features đã được scaled và encoded đúng cách.